In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
%matplotlib notebook

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Taltal,CL,2021-05-26 19:43:56,-25.4000,-70.4833,67.84,64,100,1.79
1,1,Jamestown,US,2021-05-26 19:40:29,42.0970,-79.2353,68.07,91,90,5.01
2,2,Airai,TL,2021-05-26 19:43:56,-8.9266,125.4092,55.62,88,31,3.98
3,3,Ushuaia,AR,2021-05-26 19:41:30,-54.8000,-68.3000,31.66,86,75,17.27
4,4,Albany,US,2021-05-26 19:43:27,42.6001,-73.9662,90.91,46,36,1.01


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
10,10,Contamana,PE,2021-05-26 19:43:58,-7.3333,-75.0167,86.54,58,87,3.71
14,14,Hithadhoo,MV,2021-05-26 19:44:00,-0.6000,73.0833,84.20,69,100,21.16
21,21,Santa Cruz,BO,2021-05-26 19:42:58,-17.8000,-63.1667,82.45,54,75,10.36
22,22,Nishihara,JP,2021-05-26 19:44:04,26.1842,127.7558,80.26,94,20,17.38
30,30,Hambantota,LK,2021-05-26 19:44:08,6.1241,81.1185,80.13,81,92,23.31
33,33,Anloga,GH,2021-05-26 19:44:09,5.7947,0.8973,84.42,83,18,15.59
43,43,Faanui,PF,2021-05-26 19:44:12,-16.4833,-151.7500,81.41,73,34,13.87
54,54,Bhatapara,IN,2021-05-26 19:44:16,21.7333,81.9333,84.11,56,100,5.30
55,55,Touros,BR,2021-05-26 19:44:16,-5.1989,-35.4608,85.10,76,24,17.43
76,76,Manggar,ID,2021-05-26 19:44:23,-2.8833,108.2667,82.67,79,62,8.39


In [15]:
preferred_cities_df.count()

City_ID       97
City          97
Country       97
Date          97
Lat           97
Lng           97
Max Temp      97
Humidity      97
Cloudiness    97
Wind Speed    97
dtype: int64

In [24]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Contamana,PE,86.54,-7.3333,-75.0167,
14,Hithadhoo,MV,84.20,-0.6000,73.0833,
21,Santa Cruz,BO,82.45,-17.8000,-63.1667,
22,Nishihara,JP,80.26,26.1842,127.7558,
30,Hambantota,LK,80.13,6.1241,81.1185,
33,Anloga,GH,84.42,5.7947,0.8973,
43,Faanui,PF,81.41,-16.4833,-151.7500,
54,Bhatapara,IN,84.11,21.7333,81.9333,
55,Touros,BR,85.10,-5.1989,-35.4608,
76,Manggar,ID,82.67,-2.8833,108.2667,


In [25]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [27]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel added to DF")
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel not found... skipping.
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel not found... skipping.
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel 

In [23]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Contamana,PE,86.54,-7.3333,-75.0167,Shiringal Touristic Park
14,Hithadhoo,MV,84.20,-0.6000,73.0833,Scoop Guest House
21,Santa Cruz,BO,82.45,-17.8000,-63.1667,Aviador Hotel Boutique
22,Nishihara,JP,80.26,26.1842,127.7558,Minshuku Agaihama
30,Hambantota,LK,80.13,6.1241,81.1185,Bungalow 63


In [19]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))